### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
sagemaker 2.165.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.6.0 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.4/852.4 kB 8.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 2.1 M

In [3]:
%pip install sagemaker pip --upgrade  --quiet
# Note the following may error depending on which awscli is installed in your jupyter kernel, 
# but that is ok 
%pip install ./code/botocore-1.29.157-py3-none-any.whl ./code/boto3-1.26.157-py3-none-any.whl --force

Note: you may need to restart the kernel to use updated packages.
Processing ./code/botocore-1.29.157-py3-none-any.whl
Processing ./code/boto3-1.26.157-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 3.7 MB/s eta 0:00:0000:01
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 1.3 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2

In [2]:
!rm -rf ./LLM_chatglm2_stream_model

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm2_stream_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm2-6b"
commit_hash = "b1502f4f75c71499a3d566b14463edd62620ce9f"

In [4]:
snapshot_download(repo_id=model_name, cache_dir=local_model_path)

'LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f'

### 2. 把模型拷贝到S3为后续部署做准备

In [6]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [7]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm2_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm2_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_chatglm2_stream_deploy_code
model_snapshot_path: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f


In [8]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/.gitattributes
upload: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/README.md
upload: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f/configuration_chatglm.py to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/configuration_chatglm.py
upload: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d566b14463edd62620ce9f/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/config.json
upload: LLM_chatglm2_stream_model/models--THUDM--chatglm2-6b/snapshots/b1502f4f75c71499a3d56

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [10]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118


In [11]:
!mkdir -p LLM_chatglm2_stream_deploy_code

In [35]:
%%writefile LLM_chatglm2_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer
model = None
tokenizer = None
STOP_flag = "[DONE]"

DEVICE = "cuda"
DEVICE_ID = "0"
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE
def torch_gc():
    if torch.cuda.is_available():
        with torch.cuda.device(CUDA_DEVICE):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
def load_model(properties):
    global tokenizer,model
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    model = AutoModel.from_pretrained(model_location, trust_remote_code=True).half().cuda()
    
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer



def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    for response, history in model.stream_chat(tokenizer, prompt, history=history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        this_response = response[size:]
        history = [list(h) for h in history]
        size = len(response)
        stream_buffer = { "outputs":this_response}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": history, "finished": True}
    


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    outputs.add_property("content-type", "application/jsonlines")
    outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    return outputs


Overwriting LLM_chatglm2_stream_deploy_code/model.py


In [36]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [37]:
%%writefile LLM_chatglm2_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_model/

Overwriting LLM_chatglm2_stream_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

In [38]:
%%writefile LLM_chatglm2_stream_deploy_code/requirements.txt
transformers==4.30.2

Overwriting LLM_chatglm2_stream_deploy_code/requirements.txt


In [39]:
!rm model.tar.gz
!cd LLM_chatglm2_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_chatglm2_stream_deploy_code

LLM_chatglm2_stream_deploy_code/
LLM_chatglm2_stream_deploy_code/model.py
LLM_chatglm2_stream_deploy_code/requirements.txt
LLM_chatglm2_stream_deploy_code/serving.properties


In [40]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_chatglm2_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [41]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"chatglm-stream") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

chatglm-stream-2023-08-09-06-45-35-544
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/chatglm-stream-2023-08-09-06-45-35-544


In [42]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/chatglm-stream-2023-08-09-06-45-35-544-config',
 'ResponseMetadata': {'RequestId': '1373a433-4fc1-4d6d-84f1-786c1e792b79',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1373a433-4fc1-4d6d-84f1-786c1e792b79',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Wed, 09 Aug 2023 06:45:37 GMT'},
  'RetryAttempts': 0}}

In [43]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/chatglm-stream-2023-08-09-06-45-35-544-endpoint


#### 持续检测模型部署进度

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating


### 5. 模型测试

In [22]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 2048,
  "temperature": 0.01,
  "top_p":0.8
}

CPU times: user 4.41 ms, sys: 0 ns, total: 4.41 ms
Wall time: 4.29 ms


In [23]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

In [31]:
prompts1 = """你好"""
# prompts1 = """write a 500 words story about scifiction"""
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            print(resp)
            # print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue

{'outputs': {'outputs': '你', 'finished': False}}
{'outputs': {'outputs': '好', 'finished': False}}
{'outputs': {'outputs': '👋', 'finished': False}}
{'outputs': {'outputs': '！', 'finished': False}}
{'outputs': {'outputs': '我是', 'finished': False}}
{'outputs': {'outputs': '人工智能', 'finished': False}}
{'outputs': {'outputs': '助手', 'finished': False}}
{'outputs': {'outputs': ' Chat', 'finished': False}}
{'outputs': {'outputs': 'GL', 'finished': False}}
{'outputs': {'outputs': 'M', 'finished': False}}
{'outputs': {'outputs': '2', 'finished': False}}
{'outputs': {'outputs': '-', 'finished': False}}
{'outputs': {'outputs': '6', 'finished': False}}
{'outputs': {'outputs': 'B', 'finished': False}}
{'outputs': {'outputs': '，', 'finished': False}}
{'outputs': {'outputs': '很高兴', 'finished': False}}
{'outputs': {'outputs': '见到', 'finished': False}}
{'outputs': {'outputs': '你', 'finished': False}}
{'outputs': {'outputs': '，', 'finished': False}}
{'outputs': {'outputs': '欢迎', 'finished': False}}
{'outp

#### 清除模型Endpoint和config

In [32]:
!aws sagemaker delete-endpoint --endpoint-name  {endpoint_name}

In [33]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [34]:
!aws sagemaker delete-model --model-name  {model_name}